In [1]:
import os
print([i for i in os.listdir() if 'dataset.json' in i])
print([i for i in os.listdir() if 'set_a.csv' in i])

['dataset.json']
['set_a.csv']


In [2]:
import json
import numpy as np
def DataDeserialize(jsonObj,asNumpy=False):
    toDict=json.loads(jsonObj)
    data_f=json.loads(toDict['data_f'])
    data_t=json.loads(toDict['data_t'])
    fs=json.loads(toDict['fs'])
    for key in data_f.keys():
        if asNumpy:
            data_f[key]=np.array(data_f[key]).reshape([len(data_f[key]),1])
            data_t[key]=np.array(data_t[key])
        else:
            data_f[key]=data_f[key]
            data_t[key]=data_t[key]
            fs[key]=[fs[key]]
    
    return data_f,data_t,fs

with open('dataset.json','r') as f:
    dataset=f.read()
data_dft,data_dec,fs=DataDeserialize(dataset)

In [12]:
import pandas as pd

data_labels=pd.read_csv('set_a.csv')
data_labels=data_labels[data_labels.label.notnull()]
data_labels['filename']=data_labels.fname.apply(lambda x: x[6:len(x)])
labels=data_labels.label.drop_duplicates()
data_labels=data_labels[['filename','label']]
data_labels.index=data_labels.filename

df_pd = pd.DataFrame(data_dft)
k = len(df_pd)//2

#spectra split to a halfe of fs - nyquist freq. nyquist  and PSD calculation
df_pd=df_pd.iloc[0:k,:].applymap(lambda x: np.log(x**2))

#curve normalization
for col in df_pd.columns:
    df_pd.loc[:,col]=df_pd.loc[:,col]/max(df_pd.loc[:,col])
    
df_fs = pd.DataFrame(fs,index=['fs']).transpose()
df_fs['fs'] = df_fs.fs/2
df_fs['k'] = k

df_pd=df_pd.transpose().join(data_labels).join(df_fs)
df_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,1994,1995,1996,1997,1998,1999,filename,label,fs,k
extrahls__201101070953.wav,0.716302,0.740627,0.599815,0.664514,0.786073,0.553410,0.719064,0.703307,0.607023,0.754069,...,0.631046,0.338797,0.639984,0.530173,0.377894,0.593133,extrahls__201101070953.wav,extrahls,367.5,2000
extrahls__201101091153.wav,0.739262,0.617246,0.803137,0.462727,0.489625,0.699909,0.725328,0.673239,0.572426,0.744569,...,0.664340,0.480305,0.640480,0.608836,0.620766,0.611966,extrahls__201101091153.wav,extrahls,367.5,2000
extrahls__201101152255.wav,0.913559,0.779044,0.818879,0.500293,0.702190,0.744516,0.643676,0.867989,0.750828,0.856806,...,0.602872,0.318743,0.666170,0.741681,0.656872,0.615473,extrahls__201101152255.wav,extrahls,367.5,2000
extrahls__201101160804.wav,0.695254,0.564887,0.662952,0.761360,0.728754,0.759764,0.736816,0.707393,0.781271,0.731554,...,0.597531,0.661305,0.667166,0.641461,0.588037,0.717878,extrahls__201101160804.wav,extrahls,367.5,2000
extrahls__201101160808.wav,0.822854,0.702048,0.398874,0.642867,0.578157,0.544518,0.740318,0.649023,0.726241,0.641135,...,0.572833,0.666673,0.691166,0.456296,0.648299,0.541012,extrahls__201101160808.wav,extrahls,367.5,2000


In [13]:
spark=SparkSession.builder.getOrCreate()
df=spark.createDataFrame(df_pd)

In [14]:
from pyspark.sql.functions import grouping,count
df.cube('label').agg(count('label')).show()
df.toPandas().describe()

+--------+------------+
|   label|count(label)|
+--------+------------+
|extrahls|          19|
|    null|          84|
|  normal|          31|
|  murmur|          34|
+--------+------------+



,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,fs,k
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,...,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.0,84.0
mean,0.705105,0.646034,0.643607,0.637286,0.644987,0.639547,0.627597,0.656853,0.655537,0.637170,...,0.526919,0.554309,0.553485,0.532720,0.562463,0.555915,0.558160,0.552841,367.5,2000.0
std,0.100153,0.080704,0.096546,0.089848,0.083670,0.118966,0.105078,0.087821,0.077099,0.134674,...,0.119527,0.127651,0.120348,0.133105,0.127846,0.133496,0.124976,0.137573,0.0,0.0
min,0.297124,0.360112,0.335903,0.263745,0.274716,-0.085987,0.273435,0.411791,0.393980,-0.308040,...,0.083485,0.251371,0.276954,0.186994,0.315512,0.233568,0.206821,0.133297,367.5,2000.0
25%,0.643972,0.610361,0.600994,0.602844,0.602669,0.594344,0.562949,0.625029,0.608220,0.608952,...,0.451278,0.469185,0.461060,0.438413,0.484961,0.480114,0.475005,0.476908,367.5,2000.0
50%,0.715718,0.655800,0.660470,0.646042,0.657632,0.657441,0.645353,0.659293,0.659730,0.641274,...,0.542268,0.548169,0.547114,0.525675,0.571061,0.564062,0.562868,0.556994,367.5,2000.0
75%,0.775847,0.699881,0.712271,0.701502,0.702195,0.716171,0.714607,0.707488,0.701898,0.707338,...,0.606704,0.656740,0.630929,0.624763,0.654824,0.649921,0.651132,0.648295,367.5,2000.0
max,0.913559,0.779044,0.818879,0.779574,0.790871,0.791680,0.801636,0.867989,0.830562,0.856806,...,0.730488,0.849596,0.818759,0.832410,0.870558,0.853293,0.880259,0.880190,367.5,2000.0


In [15]:
df_pd.shape
df.write.save('dataframe.parquet',format='parquet',mode='overwrite')